In [ ]:
#https://www.kaggle.com/boojum/connecting-voxel-spaces/

I thought that all pipelines presented on public notebooks are giving random output, so I decided to think a little bit differently. The idea is to extract features using MRI images and tumor mask. Here I'm giving an example for one image, maybe I'm doing something wrong, but I'll be glad to get feedback from the community. Also I'm worried that it will be out of time on the whole test set

### Registration 
https://www.kaggle.com/boojum/connecting-voxel-spaces/
![](https://sun9-45.userapi.com/impg/Flbnug2OUli1ecXsoIKeUasIGXGj_5hqjX4cRg/z2nfz8-b3a0.jpg?size=2560x1153&quality=96&sign=0536543610f1655d967af88dbc775e98&type=album)

### Segmentation 
Unet
https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/

### Features

https://pyradiomics.readthedocs.io/en/latest/features.html#module-radiomics.shape2D

In [ ]:
!pip install pyradiomics

In [ ]:
import os
import sys 
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image
import pydicom
import torch
import nibabel as nib
import matplotlib.pyplot as plt
import SimpleITK as sitk
import radiomics

train_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'

In [ ]:
train_dirs = sorted(os.listdir(train_path))

In [ ]:
reader = sitk.ImageSeriesReader()
reader.LoadPrivateTagsOn()

In [ ]:
def resample(image, ref_image):

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    
    resampler.SetTransform(sitk.AffineTransform(image.GetDimension()))

    resampler.SetOutputSpacing(ref_image.GetSpacing())

    resampler.SetSize(ref_image.GetSize())

    resampler.SetOutputDirection(ref_image.GetDirection())

    resampler.SetOutputOrigin(ref_image.GetOrigin())

    resampler.SetDefaultPixelValue(image.GetPixelIDValue())

    resamped_image = resampler.Execute(image)
    
    return resamped_image

In [ ]:
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
%%time
def get_img(index):
    filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[index]}/T1w')
    reader.SetFileNames(filenamesDICOM)
    t1_sitk = reader.Execute()

    filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[index]}/FLAIR')
    reader.SetFileNames(filenamesDICOM)
    flair_sitk = reader.Execute()

    filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[index]}/T1wCE')
    reader.SetFileNames(filenamesDICOM)
    t1wce_sitk = reader.Execute()

    flair_resampled = resample(flair_sitk, t1_sitk)
    t1wce_resampled = resample(t1wce_sitk, t1_sitk)

    t1_sitk_array = normalize(sitk.GetArrayFromImage(t1_sitk))
    flair_resampled_array = normalize(sitk.GetArrayFromImage(flair_resampled))
    t1wce_resampled_array = normalize(sitk.GetArrayFromImage(t1wce_resampled))

    stacked = np.stack([t1_sitk_array, flair_resampled_array, t1wce_resampled_array])

    to_rgb = stacked[:,t1_sitk_array.shape[0]//2,:,:].transpose(1,2,0)
    im = Image.fromarray((to_rgb * 255).astype(np.uint8))
    return im

In [ ]:
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)

In [ ]:
im = get_img(3)

In [ ]:
test_img = np.array([np.moveaxis(np.array(im.resize((256, 256))), -1, 0)])
test_res = model(torch.Tensor(test_img))

In [ ]:
plt.imshow(test_img[0, 1, :, :])

In [ ]:
plt.imshow(test_res[0][0].detach().cpu().numpy() > 0.5)

In [ ]:
shape = radiomics.shape2D.RadiomicsShape2D(
    sitk.GetImageFromArray(test_img), 
    sitk.GetImageFromArray(np.array([
        test_res[0][0].detach().cpu().numpy() > 0.5
    ]).astype(np.uint8)),
    force2D=True
)

In [ ]:
shape.getMeshSurfaceFeatureValue(), \
shape.getPixelSurfaceFeatureValue(),\
shape.getPerimeterFeatureValue(), \
shape.getPerimeterSurfaceRatioFeatureValue(), \
shape.getSphericityFeatureValue(), \
shape.getSphericalDisproportionFeatureValue(), \
shape.getMaximumDiameterFeatureValue(), \
shape.getMajorAxisLengthFeatureValue(), \
shape.getMinorAxisLengthFeatureValue(), \
shape.getElongationFeatureValue(), \

## P.S. Warning
The segmentation doesn't not working properly on all images due to the different tumor modalities. The model was trained on low-grade tumors, so be aware. Let's see the examples

#### For this type of broken segmentation like in '00009' we could perform a little trick to fix. 
We need to multiply mask to the empty space of the original image

In [ ]:
im = get_img(6)
test_img = np.array([np.moveaxis(np.array(im.resize((256, 256))), -1, 0)])
test_res = model(torch.Tensor(test_img))

f, axarr = plt.subplots(1,5, figsize=(20, 20))
axarr[0].imshow(test_img[0, 0])
axarr[1].imshow(test_img[0, 1])
axarr[2].imshow(test_img[0, 2])
axarr[3].imshow(test_res[0][0].detach().cpu().numpy() > 0.5)
axarr[4].imshow((test_res[0][0].detach().cpu().numpy() > 0.5) * (test_img[0, 1] != 0))

#### For the "00003" image the broken segmentation could be due to the bad registration and this trick won't work

In [ ]:
im = get_img(2)
test_img = np.array([np.moveaxis(np.array(im.resize((256, 256))), -1, 0)])
test_res = model(torch.Tensor(test_img))

f, axarr = plt.subplots(1,5, figsize=(20, 20))
axarr[0].imshow(test_img[0, 0])
axarr[1].imshow(test_img[0, 1])
axarr[2].imshow(test_img[0, 2])
axarr[3].imshow(test_res[0][0].detach().cpu().numpy() > 0.5)
axarr[4].imshow((test_res[0][0].detach().cpu().numpy() > 0.5) * (test_img[0, 1] != 0))

#### Let's see more examples of broken segmentation

In [ ]:
im = get_img(9)
test_img = np.array([np.moveaxis(np.array(im.resize((256, 256))), -1, 0)])
test_res = model(torch.Tensor(test_img))

f, axarr = plt.subplots(1,5, figsize=(20, 20))
axarr[0].imshow(test_img[0, 0])
axarr[1].imshow(test_img[0, 1])
axarr[2].imshow(test_img[0, 2])
axarr[3].imshow(test_res[0][0].detach().cpu().numpy() > 0.5)
axarr[4].imshow((test_res[0][0].detach().cpu().numpy() > 0.5) * (test_img[0, 1] != 0))

In [ ]:
im = get_img(20)
test_img = np.array([np.moveaxis(np.array(im.resize((256, 256))), -1, 0)])
test_res = model(torch.Tensor(test_img))

f, axarr = plt.subplots(1,5, figsize=(20, 20))
axarr[0].imshow(test_img[0, 0])
axarr[1].imshow(test_img[0, 1])
axarr[2].imshow(test_img[0, 2])
axarr[3].imshow(test_res[0][0].detach().cpu().numpy() > 0.5)
axarr[4].imshow((test_res[0][0].detach().cpu().numpy() > 0.5) * (test_img[0, 1] != 0))

In [ ]:
im = get_img(24)
test_img = np.array([np.moveaxis(np.array(im.resize((256, 256))), -1, 0)])
test_res = model(torch.Tensor(test_img))

f, axarr = plt.subplots(1,5, figsize=(20, 20))
axarr[0].imshow(test_img[0, 0])
axarr[1].imshow(test_img[0, 1])
axarr[2].imshow(test_img[0, 2])
axarr[3].imshow(test_res[0][0].detach().cpu().numpy() > 0.5)
axarr[4].imshow((test_res[0][0].detach().cpu().numpy() > 0.5) * (test_img[0, 1] != 0))